In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
nash_list = pd.read_csv('../data/listings.csv')

In [27]:
nash_list

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6422,https://www.airbnb.com/rooms/6422,20250315060229,2025-03-15,city scrape,Nashville Charm,30 day plus rental - book for one month and th...,Historic East Nashville is home to many new an...,https://a0.muscache.com/pictures/miso/Hosting-...,12172,...,4.96,4.92,4.98,NaN,f,1,0,1,0,3.46
1,39870,https://www.airbnb.com/rooms/39870,20250315060229,2025-03-15,city scrape,Close to Vanderbilt 2,"Since I am older, I need for guests to be vacc...","The house is in a safe, quiet, ""college"" neig...",https://a0.muscache.com/pictures/miso/Hosting-...,171184,...,4.97,4.93,4.91,NaN,f,1,0,1,0,5.22
2,59576,https://www.airbnb.com/rooms/59576,20250315060229,2025-03-15,city scrape,Large Main Suite near Lake *ladies only NS plz,non-smokers only please<br /><br />furnished r...,We live on a cul-de-sac in a small community o...,https://a0.muscache.com/pictures/4e4e34db-3155...,812128,...,5.00,5.00,5.00,NaN,f,6,1,5,0,0.09
3,72906,https://www.airbnb.com/rooms/72906,20250315060229,2025-03-15,city scrape,Vandy/Belmont/10 mins to Broadway - Sunny 800 ...,Entire private top floor of my home with share...,Historic Belmont-Hillsboro neighborhood. Walk-...,https://a0.muscache.com/pictures/1777db4f-d0a6...,176117,...,4.99,4.97,4.90,NaN,f,1,1,0,0,4.50
4,258817,https://www.airbnb.com/rooms/258817,20250315060229,2025-03-15,city scrape,"ButterflyRoom-queen room, private bath",Morningstar House! Extended stay - 3+ months- ...,"I'm very near the Percy Priest Lake, natural ...",https://a0.muscache.com/pictures/eef19bc4-2743...,22296,...,4.91,4.74,4.73,NaN,f,8,0,7,1,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9580,1376225002885392158,https://www.airbnb.com/rooms/1376225002885392158,20250315060229,2025-03-15,city scrape,Gorgeous New Nash Home | Rooftop Views + Hot Tub,Experience our beautifully designed BRAND NEW ...,The home is located on the north side of Nashv...,https://a0.muscache.com/pictures/prohost-api/H...,409764479,...,NaN,NaN,NaN,NaN,t,2,2,0,0,NaN
9581,1376227174119190675,https://www.airbnb.com/rooms/1376227174119190675,20250315060229,2025-03-15,city scrape,"Hot Tub, Poker, Pool Table & Rooftop City MLS ...","Welcome to ""The Gambler,"" your luxury Nashvill...",Located near the Geodis Park MLS Soccer Stadiu...,https://a0.muscache.com/pictures/prohost-api/H...,2442207,...,NaN,NaN,NaN,NaN,t,37,37,0,0,NaN
9582,1376286726988679347,https://www.airbnb.com/rooms/1376286726988679347,20250315060229,2025-03-15,city scrape,Soprano - Private Rooftop - 8 mins to Broadway,"Welcome to Soprano, a luxurious 3-story townho...",Berry Hill,https://a0.muscache.com/pictures/prohost-api/H...,531829128,...,NaN,NaN,NaN,NaN,t,4,4,0,0,NaN
9583,1376305384125885719,https://www.airbnb.com/rooms/1376305384125885719,20250315060229,2025-03-15,city scrape,Golden Luxe Suite | DT Nash,Unwind in this stylish one-bedroom suite in th...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,542795691,...,NaN,NaN,NaN,NaN,t,5,5,0,0,NaN


In [6]:
nash_review = pd.read_csv('../data/reviews.csv')

In [7]:
nash_review

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3907104,19294916,2014-09-09,4188194,Patrick,Alan's place was great to stay in for my first...
1,3907104,20792132,2014-10-05,3931094,Erim,This is the absolute best location for a local...
2,3907104,21010270,2014-10-09,14795914,Scott,We had a great stay at Alan's place in East Na...
3,3907104,21325023,2014-10-14,1550762,Tyler,"We loved Alan's place! Very comfy, quiet, clea..."
4,3907104,22039991,2014-10-28,22330921,Patrick,Alan did an incredible job in making my family...
...,...,...,...,...,...,...
748008,1364763966554115862,1368307933515891773,2025-03-02,326180073,Ashley,This was our second time booking with Mohit. T...
748009,1364763966554115862,1371194981191229418,2025-03-06,154561022,Ben,Mohit was an excellent host. Great communicati...
748010,1364791864293670455,1371850810461734529,2025-03-07,418741620,Joshua,"I was in Nashville for work , it was awesome w..."
748011,1365671040067474810,1374018904128956468,2025-03-10,552895380,福丽,"Alex is very nice , it is rarely to see a sing..."


In [8]:
len(set(nash_review))

6

In [25]:
nash_neighbors = pd.read_csv('../data/neighbourhoods.csv')
nash_neighbors

,neighbourhood_group,neighbourhood
0,NaN,District 1
1,NaN,District 10
2,NaN,District 11
3,NaN,District 12
4,NaN,District 13
5,NaN,District 14
6,NaN,District 15
7,NaN,District 16
8,NaN,District 17
9,NaN,District 18
